In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
seq_len = 8

In [3]:
user1 = pd.read_csv('user1_preprocessed_2.csv')
user2 = pd.read_csv('user2_preprocessed_2.csv')

user1 = user1.set_index(pd.DatetimeIndex(user1['Time']))
user2 = user2.set_index(pd.DatetimeIndex(user2['Time']))

user1.drop(columns='Time', inplace=True)
user2.drop(columns='Time', inplace=True)

print(user1.head(5))
print(user1.shape)

                           App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                           
2021-09-10 11:30:00  TextLogger.exe              784.401936            296.0   
2021-09-10 12:00:00    explorer.exe              784.401936            296.0   
2021-09-10 12:30:00       opera.exe              618.809407           1048.0   
2021-09-10 13:00:00       opera.exe             1048.829218            280.0   
2021-09-10 13:30:00       opera.exe             1219.861964            348.0   

                     Speed(ms)  PAM_Val  
Time                                     
2021-09-10 11:30:00   2.290322      2.0  
2021-09-10 12:00:00   6.713206      2.0  
2021-09-10 12:30:00   7.769313      1.0  
2021-09-10 13:00:00  10.621018      1.0  
2021-09-10 13:30:00   5.499976      2.0  
(40, 5)


In [4]:
#print(user1.App_Name.value_counts())
#print(user2.App_Name.value_counts())

#factor = pd.factorize(pd.concat([user1['App_Name'], user2['App_Name']]))
#user1.App_Name = factor[0][:len(user1)]
#user2.App_Name = factor[0][len(user1):]

#print(user1.App_Name.value_counts())
#print(user2.App_Name.value_counts())

In [5]:
print(user1.head(5))

                           App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                           
2021-09-10 11:30:00  TextLogger.exe              784.401936            296.0   
2021-09-10 12:00:00    explorer.exe              784.401936            296.0   
2021-09-10 12:30:00       opera.exe              618.809407           1048.0   
2021-09-10 13:00:00       opera.exe             1048.829218            280.0   
2021-09-10 13:30:00       opera.exe             1219.861964            348.0   

                     Speed(ms)  PAM_Val  
Time                                     
2021-09-10 11:30:00   2.290322      2.0  
2021-09-10 12:00:00   6.713206      2.0  
2021-09-10 12:30:00   7.769313      1.0  
2021-09-10 13:00:00  10.621018      1.0  
2021-09-10 13:30:00   5.499976      2.0  


In [6]:
def convert_to_list_of_trajectories(df, seq_len):
    timedelta = pd.Timedelta(minutes=30)

    currDatetime = df.index[0]
    startDatetime = currDatetime
    endDatetime = currDatetime

    # split into trajectories
    trajectories = []
    while currDatetime != df.index[-1] + timedelta:
        if currDatetime + timedelta not in df.index:
            if currDatetime in df.index:
                endDatetime = currDatetime

                trajectories.append(df[startDatetime : endDatetime])

                currDatetime += timedelta
                startDatetime = currDatetime
            else:
                currDatetime += timedelta
                startDatetime = currDatetime
        else:
            currDatetime += timedelta

    print([len(trajectory) for trajectory in trajectories])

    data = []
    for trajectory in trajectories:
        temp_data = []
        # Cut data by sequence length
        for i in range(0, len(trajectory) - seq_len):
            _x = trajectory[i:i + seq_len]
            temp_data.append(_x)
        data += temp_data

    # Mix the datasets (to make it similar to i.i.d)
    idx = np.random.permutation(len(data))
    processed_data = []
    for i in range(len(data)):
        processed_data.append(data[idx[i]])

    return processed_data
    #return data

user1_traj = convert_to_list_of_trajectories(user1, seq_len)
user2_traj = convert_to_list_of_trajectories(user2, seq_len)

[15, 8, 17]
[22, 18, 17, 9, 13, 15]


In [7]:
#user1_temp = []
#for index, traj in enumerate(user1_traj):
#    if traj.PAM_Val.nunique() > 2:
#        user1_temp.append(traj)

#user2_temp = []
#for index, traj in enumerate(user2_traj):
#    if traj.PAM_Val.nunique() > 2:
#        user2_temp.append(traj)


# ==============
def filter_to_minority(user_traj):
    user_temp = []
    for index, traj in enumerate(user_traj):
        minority_count = 0
        val_counts = traj.PAM_Val.value_counts().sort_index()
        #print(val_counts)
        if 3 in val_counts.index:
            minority_count += val_counts.loc[3]
        if 4 in val_counts.index:
            minority_count += val_counts.loc[4]
        if minority_count >= 2:
            user_temp.append(traj)

    return user_temp

#user1_temp = filter_to_minority(user1_traj)
#user2_temp = filter_to_minority(user2_traj)

# ==============

#print(len(user1_traj), len(user1_temp))
#print(len(user2_traj), len(user2_temp))

#user1_traj = user1_temp
#user2_traj = user2_temp

print(len(user1_traj))
print(len(user2_traj))

16
46


In [8]:
user1 = pd.concat(user1_traj)
user2 = pd.concat(user2_traj)

In [9]:
print(user1.PAM_Val.value_counts())

2.0    52
1.0    44
3.0    16
4.0    16
Name: PAM_Val, dtype: int64


In [10]:
user_comb = pd.concat([user1, user2])
print(user_comb)

                         App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                         
2021-09-10 13:30:00     opera.exe             1219.861964            348.0   
2021-09-10 14:00:00     opera.exe              663.780873            262.0   
2021-09-10 14:30:00     opera.exe              871.821915            458.0   
2021-09-10 15:00:00     opera.exe             1585.753779             57.0   
2021-09-10 15:30:00     opera.exe             1034.388266            171.0   
...                           ...                     ...              ...   
2021-09-09 11:00:00  explorer.exe             2482.386615             31.0   
2021-09-09 11:30:00  explorer.exe             2309.364528             87.0   
2021-09-09 12:00:00  explorer.exe             2966.810912            197.0   
2021-09-09 12:30:00  explorer.exe             2442.706585            361.0   
2021-09-09 13:00:00  explorer.exe             2038.423218       

In [11]:
user_comb = user_comb.assign(example_id = [index // seq_len for index in range(len(user_comb))])
print(user_comb)

                         App_Name  Inactivity Duration(s)  Keystroke_Count  \
Time                                                                         
2021-09-10 13:30:00     opera.exe             1219.861964            348.0   
2021-09-10 14:00:00     opera.exe              663.780873            262.0   
2021-09-10 14:30:00     opera.exe              871.821915            458.0   
2021-09-10 15:00:00     opera.exe             1585.753779             57.0   
2021-09-10 15:30:00     opera.exe             1034.388266            171.0   
...                           ...                     ...              ...   
2021-09-09 11:00:00  explorer.exe             2482.386615             31.0   
2021-09-09 11:30:00  explorer.exe             2309.364528             87.0   
2021-09-09 12:00:00  explorer.exe             2966.810912            197.0   
2021-09-09 12:30:00  explorer.exe             2442.706585            361.0   
2021-09-09 13:00:00  explorer.exe             2038.423218       

In [12]:
user_comb.to_csv('./preprocessed_gretel.csv')